In [65]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

In [67]:
# 경로는 너가 저장한 위치에 맞게 수정!
with open('model_AB_lgbm.dat', 'rb') as f:
    model_AB = pickle.load(f)

with open('model_CDE_tuned_lgbm.dat', 'rb') as f:
    model_CDE = pickle.load(f)

with open('gate_model_lgbm.dat', 'rb') as f:
    gate_model = pickle.load(f)

with open('label_encoder_AB.dat', 'rb') as f:
    le_ab = pickle.load(f)

with open('label_encoder_CDE.dat', 'rb') as f:
    le_cde = pickle.load(f)

In [69]:
# 예측할 데이터를 읽어온다.
X_test = pd.read_parquet('test_all_high_corr.parquet')

In [71]:
id_col = X_test['ID'].copy()

In [73]:
X_test = X_test.drop(columns=['ID'])

In [75]:
# 3. 게이트 예측: P(AB), P(CDE)
# ─────────────────────────────
p_ab_gate  = gate_model.predict_proba(X_test)[:, 1]
p_cde_gate = 1 - p_ab_gate

# ─────────────────────────────
# 4. 서브모델 예측 확률
# ─────────────────────────────
proba_ab  = model_AB.predict_proba(X_test)
proba_cde = model_CDE.predict_proba(X_test)

# ─────────────────────────────
# 5. 클래스 순서 맞추기
# ─────────────────────────────
idx_A = le_ab.transform(['A'])[0]
idx_B = le_ab.transform(['B'])[0]
idx_C = le_cde.transform(['C'])[0]
idx_D = le_cde.transform(['D'])[0]
idx_E = le_cde.transform(['E'])[0]

# ─────────────────────────────
# 6. 조건부 결합 확률 계산
# ─────────────────────────────
p_final = np.column_stack([
    proba_ab[:, idx_A] * p_ab_gate,
    proba_ab[:, idx_B] * p_ab_gate,
    proba_cde[:, idx_C] * p_cde_gate,
    proba_cde[:, idx_D] * p_cde_gate,
    proba_cde[:, idx_E] * p_cde_gate
])
p_final /= p_final.sum(axis=1, keepdims=True)

# ─────────────────────────────
# 7. 최종 클래스 예측
# ─────────────────────────────
label_order = np.array(['A', 'B', 'C', 'D', 'E'])
y_pred_final = label_order[p_final.argmax(axis=1)]

# ─────────────────────────────
# 8. 예측 결과 붙이기
# ─────────────────────────────
X_test_with_segment = X_test.copy()
X_test_with_segment['ID'] = id_col
X_test_with_segment['Segment'] = y_pred_final

# ─────────────────────────────
# 9. 저장
# ─────────────────────────────
X_test_with_segment.to_csv('result/test_with_segment.csv', index=False)
print("✅ Segment 예측 완료! 'result/test_with_segment.csv'에 저장됨.")

✅ Segment 예측 완료! 'result/test_with_segment.csv'에 저장됨.
